# Загрузка Pandas и очистка данных

In [15]:
import pandas as pd
import re
from datetime import datetime

In [16]:
df = pd.read_csv('main_task_new.csv')

In [53]:
df = pd.read_csv('main_task_new.csv')

# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

# Функция для превращения строки в список в поле "кухни"
def cuisines_to_list(string):
    li = list(string[2:-2].split("', '"))
    return li

# Заполняем пропуски строкой " None, "
df['Cuisine Style'] = df['Cuisine Style'].fillna("  None, ")

# Превращаем кухни из str в list
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: cuisines_to_list(x))

# Считаем по каждому ресторану количество кухонь
df['Restaurant Cusines Count'] = df['Cuisine Style'].apply(lambda x: len(x))

# Сначала разделим отзывы на отзыв и дату отзыва

def split_reviews(string):
    li = list(string[2:-2].split("], ["))
    return li

df['Reviews'] = df['Reviews'].apply(lambda x: split_reviews(x))

# Превратим текст отзывов и дату в список

def f_reviews(rev):
    rev_new = []
    
    for elem in rev:
        rev_new.append(elem[1:-1].split("', '"))
    
    return rev_new


def rev_to_datetime(rev):
    rev_new = []
    rev_date = []
    
    for elem in rev[1]:
        try:
            rev_date.append(datetime.strptime(elem, '%m/%d/%Y'))
        except(ValueError, TypeError):
            continue
            
    rev_new.append(rev[0])
    rev_new.append(rev_date)
    
    return rev_new
    

df['Reviews'] = df['Reviews'].apply(lambda x: f_reviews(x))
df['Reviews'] = df['Reviews'].apply(lambda x: rev_to_datetime(x))

# Создадим dummy_variables из City
df = pd.concat([df.drop('City', axis=1), pd.get_dummies(df['City'], drop_first=True)], axis=1)

# Создадим dummy_variables из Price Range
df['Price Range'] = df['Price Range'].fillna("NotSet")
df = pd.concat([df.drop('Price Range', axis=1), pd.get_dummies(df['Price Range'], drop_first=True)], axis=1)

In [54]:
df = df[['Restaurant_id', 'Ranking', 'Rating', 'Number of Reviews', 'Restaurant Cusines Count']]
df = df.fillna(0)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [55]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [56]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [57]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [58]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [59]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [60]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.4138453442460317
